# Package Example

## Imports

In [ ]:
from plants_sm.data_structures.dataset.multi_input_dataset import MultiInputDataset
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.data_standardization.compounds.deepmol_standardizers import DeepMolStandardizer
from plants_sm.featurization.proteins.bio_embeddings.word2vec import Word2Vec
from plants_sm.featurization.compounds.deepmol_descriptors import DeepMolDescriptors
from torch import nn
from sklearn.metrics import f1_score
from torch.optim import Adam
from plants_sm.models.constants import BINARY
from plants_sm.models.pytorch_model import PyTorchModel
from plants_sm.models.baseline_model import BaselineModel
from plants_sm.tokenisation.compounds.smilespe import AtomLevelTokenizer
import pandas as pd


from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [ ]:
pd.read_csv(multi_input_dataset_csv).head()

## Read datasets

In [ ]:
multi_input_dataset_csv = "./data/aminotransferase_binary.csv"

multi_input_dataset = MultiInputDataset.from_csv(file_path=multi_input_dataset_csv,
                                                 representation_fields={"proteins": "SEQ",
                                                                       "ligands": "SUBSTRATES"},
                                                 instances_ids_field={"interaction": "ids"},
                                                 labels_field="LogSpActivity")

The MultiInputDataset converts everything into dictionaries to ensure that we are always processing unique sequences and not the same multiple times

In [ ]:
multi_input_dataset.instances["proteins"]

In [ ]:
multi_input_dataset.instances["ligands"]

## Standardize sequences

In [ ]:
multi_input_dataset = ProteinStandardizer(n_jobs=50).fit_transform(multi_input_dataset, instance_type="proteins")

In [ ]:
multi_input_dataset.instances["proteins"]

In [ ]:
multi_input_dataset = DeepMolStandardizer(n_jobs=50).fit_transform(multi_input_dataset, "ligands")

In [ ]:
multi_input_dataset.instances["ligands"]

## Generating features and encodings

In [ ]:
proteins_one_hot = Word2Vec().fit(multi_input_dataset, "proteins")
multi_input_dataset = proteins_one_hot.transform(multi_input_dataset, "proteins")

In [ ]:
deepmol_wrapper = DeepMolDescriptors(n_jobs=8).fit(multi_input_dataset, instance_type="ligands")
multi_input_dataset = deepmol_wrapper.transform(multi_input_dataset, instance_type="ligands")

In [ ]:
multi_input_dataset.X["proteins"]

In [ ]:
multi_input_dataset.X["ligands"]

In [ ]:
multi_input_dataset.y

## Tokenizers

In [ ]:
AtomLevelTokenizer().tokenize("CC[C@H](C)[C@@H](C(=O)O)N")

## Running models

In [ ]:
input_size_proteins = multi_input_dataset.X["proteins"].shape[1]
input_size_compounds = multi_input_dataset.X["ligands"].shape[1]
model = BaselineModel(input_size_proteins, input_size_compounds, hidden_layers_proteins=[500, 500],
                      hidden_layers_compounds=[500, 500], hidden_layers_interaction=[500, 500])

wrapper = PyTorchModel(model=model, loss_function=nn.BCELoss(),
                               validation_metric=f1_score,
                               problem_type=BINARY, batch_size=50, epochs=100,
                               optimizer=Adam(model.parameters(), lr=0.0001), progress=50,
                               logger_path="small_dataset.log")
wrapper.fit(multi_input_dataset)